In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def read_stop_words(stop_words_dict):
    return set(word.strip() for word in open(stop_words_dict ))

stop_words = read_stop_words("stop_words_en.txt")

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total Words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop Words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
%%writefile stats.py

import sys
import re

for line in sys.stdin:
    try:
        counter, count =line.strip().split('=', 1)
        count = int(count)
        if counter == 'Stop Words':
            stop_count = count
        elif counter == 'Total Words':
            total_count = count        
    except ValueError as e:
        continue
    #print("%s\t%d" % (counter, count))
print("%.2f" % (stop_count/float(total_count)))

Writing stats.py


In [5]:
!hdfs dfs -mkdir /data/wiki/gaio_files

In [6]:
! hdfs dfs -ls /data/wiki/gaio_files

In [7]:
!hdfs dfs -ls /data/wiki/en_articles_part/articles-part/

-rwxrwxrwx   1 jovyan supergroup   76861985 2017-10-17 13:15 /data/wiki/en_articles_part/articles-part


In [8]:
!hdfs dfs -cat /data/wiki/en_articles_part/articles-part | head > a.txt

cat: Unable to write to output stream.


In [9]:
%ls

20180514-stopword-SMALL-WORK.ipynb  mapper.py   stats.py
a.txt                               README.md   supervisord.log
DemoNotebook.ipynb                  reducer.py  supervisord.pid


In [10]:
!hdfs dfs -copyFromLocal a.txt /data/wiki/gaio_files

In [11]:
! hdfs dfs -ls /data/wiki/gaio_files

Found 1 items
-rw-r--r--   1 jovyan supergroup     424525 2018-05-18 03:55 /data/wiki/gaio_files/a.txt


In [12]:
! hdfs dfs -ls /datasets/stop_words_en.txt

-rw-r--r--   1 jovyan supergroup       1914 2017-10-17 13:16 /datasets/stop_words_en.txt


In [13]:
%%bash

OUT_DIR="stop_words_result_"$(date +"%s%6N")
NUM_REDUCERS=8
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/gaio_files \
    -output ${OUT_DIR} > /dev/null 2> $LOGS

#hdfs dfs -cat ${OUT_DIR}/part-00000 | head

chmod 777 ./stats.py
cat ${LOGS} | grep -e 'Total Words' -e 'Stop Words' | python ./stats.py

Stop Words	24349
Total Words	64795
0.38


rm: `stop_words_result_1526615719297953': No such file or directory


In [15]:
!cat stderr_logs.txt

Exception in thread "main" java.io.FileNotFoundException: File /datasets/stop_words_en.txt’ does not exist.
	at org.apache.hadoop.util.GenericOptionsParser.validateFiles(GenericOptionsParser.java:406)
	at org.apache.hadoop.util.GenericOptionsParser.processGeneralOptions(GenericOptionsParser.java:313)
	at org.apache.hadoop.util.GenericOptionsParser.parseGeneralOptions(GenericOptionsParser.java:488)
	at org.apache.hadoop.util.GenericOptionsParser.<init>(GenericOptionsParser.java:170)
	at org.apache.hadoop.util.GenericOptionsParser.<init>(GenericOptionsParser.java:153)
	at org.apache.hadoop.util.ToolRunner.run(ToolRunner.java:70)
	at org.apache.hadoop.util.ToolRunner.run(ToolRunner.java:90)
	at org.apache.hadoop.streaming.HadoopStreaming.main(HadoopStreaming.java:50)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImp

In [23]:
!hdfs dfs -cat wordcount_rev_1526324102803277/part-00000 | head -7 | tail -1

is	584
cat: Unable to write to output stream.


In [15]:
%ls

20180514-wordcount-reverse-WORK.ipynb  README.md      supervisord.log
a.txt                                  reducer.py     supervisord.pid
DemoNotebook.ipynb                     rx_mapper.py
mapper.py                              rx_reducer.py
